<a href="https://colab.research.google.com/github/agaonsindhe/search-agents/blob/main/ACI_Assignment_1_SolutionTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. Anshuman Gaonsindhe - 2023ab05150
2. Amit Kumar - 2023aa05262
3. Karamveer Singh - 2023aa05649
4. Vishal Pankaj - 2023aa05260 
5. Ravi Sharma - 2023aa05622

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [1]:
#Code Block : Set Initial State (Must handle dynamic inputs)
import math
from queue import PriorityQueue

# List of cities based on the given problem
cities_list = ["Panaji", "Raichur", "Mangalore", "Bellari", "Tirupathi", "Kurnool", "Kozhikode", "Bangalore", "Nellore", "Chennai"]

#Cities info with latitude and longitude
cities_info = {
    "Panaji": (15.4909,73.8278),
    "Raichur": (16.2076,77.3463),
    "Mangalore": (12.9141,74.8560),
    "Bellari": (15.1394,76.9214),
    "Tirupathi": (13.6288, 79.4192),
    "Kurnool": (15.8281,78.0373),
    "Kozhikode": (11.2588,75.7804),
    "Bangalore": (12.9716,77.5946),
    "Nellore": (14.4426,79.9865),
    "Chennai":(13.0827,80.2707)
}

# Map of cities with their successor cities.
graph = {
    "Panaji": {"Mangalore": 365 , "Raichur": 457},
    "Mangalore": {"Panaji": 365, "Bangalore": 352, "Kozhikode": 233},
    "Bangalore": {"Mangalore": 352, "Kozhikode": 356, "Bellari": 153, "Chennai": 346},
    "Kozhikode": {"Mangalore": 233, "Bangalore": 356},
    "Tirupathi": {"Bellari": 379, "Raichur": 453, "Chennai": 153},
    "Nellore": {"Kurnool": 325, "Chennai": 175},
    "Chennai": {"Tirupathi": 153, "Nellore": 175,"Bangalore": 346},
    "Raichur": {"Panaji": 457,"Tirupathi": 453,"Kurnool": 100},
    "Bellari": {"Bangalore": 153,"Tirupathi": 379},
    "Kurnool": {"Nellore": 325,"Raichur": 100}
}

# Distances between cities as provided in the graph
distances = {
    ('Panaji', 'Mangalore'): 365,
    ('Mangalore', 'Panaji'): 365,
    ('Panaji', 'Raichur'): 457,
    ('Raichur', 'Panaji'): 457,
    ('Raichur', 'Tirupathi'): 453,
    ('Tirupathi', 'Raichur'): 453,
    ('Raichur', 'Kurnool'): 100,
    ('Kurnool', 'Raichur'): 100,
    ('Mangalore', 'Kozhikode'): 233,
    ('Kozhikode', 'Mangalore'): 233,
    ('Mangalore', 'Bangalore'): 352,
    ('Bangalore', 'Mangalore'): 352,
    ('Kozhikode', 'Bangalore'): 356,
    ('Bangalore', 'Kozhikode'): 356,
    ('Bangalore', 'Chennai'): 346,
    ('Chennai', 'Bangalore'): 346,
    ('Bangalore', 'Bellari'): 153,
    ('Bellari', 'Bangalore'): 153,
    ('Bellari', 'Tirupathi'): 379,
    ('Tirupathi', 'Bellari'): 379,
    ('Tirupathi', 'Chennai'): 153,
    ('Chennai', 'Tirupathi'): 153,
    ('Kurnool', 'Nellore'): 325,
    ('Nellore', 'Kurnool'): 325,
    ('Nellore', 'Chennai'): 175,
    ('Chennai', 'Nellore'): 175
}

# Function to set start and goal city, with dynamic input.
def set_initial_state(start_city=None, end_city=None):
    """
    Sets the initial state for the traveling problem with dynamic inputs.

    :param start_city: The starting city for the travel.
    :param end_city: The destination city for the travel.
    :return: A dictionary representing the initial state of the problem.
    """
    initial_state = {
        'start_city': start_city,
        'end_city': end_city,
    }

    return initial_state


import random
# Function to calculate valid path for initial population.
def generate_valid_path(start_city, goal_city, graph):
    """
    Generate a valid path from start to city to goal city based on map connections.
    
    :param start_city: The starting city for the path.
    :param goal_city: The goal city for the path.
    :param graph: A dictionary or similar structure representing valid connections between cities.
    :return: A list representing a valid path.
    """
    path = [start_city]
    current_city = start_city
    while current_city != goal_city:
        # Get valid next steps from current city
        next_cities = [city for city in graph[current_city] if city not in path]
        if not next_cities:  # No valid next city.
            break
        next_city = random.choice(next_cities)
        path.append(next_city)
        current_city = next_city
    return path

# Function to create the initial state for ga: a population of paths
def set_initial_state_for_ga(start, end, cities, population_size):
    # Initialize the population with random valid paths
    population = []
    for _ in range(population_size):
        path = generate_valid_path(start, end, cities)
        population.append(path)
    return population

# Example usage
# initial_state = set_initial_state(start_city='Panaji', end_city='Chennai')
# print(initial_state)


In [2]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

import math

# Funnction calculating distance based on latitude and logitude in kms.
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of earth in kilometers.
    return c * r

# Function to calculate cost matrix of each city.
def set_transition_and_cost_matrix(cities_info):
    """
    Initializes a matrix/dictionary representing the graph of cities with the cost of travel between them.

    :param cities_info: A dictionary with city names as keys and their lat, long as values.
    :return: A dictionary of dictionaries representing the distances between cities.
    """
    matrix = {}
    for city, (lat1, lon1) in cities_info.items():
        matrix[city] = {}
        for destination, (lat2, lon2) in cities_info.items():
            if city != destination: # Avoid distance to itself
                distance = haversine(lat1, lon1, lat2, lon2)
                matrix[city][destination] = distance
    return matrix

# Function calculating heuristic value from all the nodes to the goal city city node.
def get_heuristics(goal_city):
    heuristics = {}
    for k in cities_info.keys():
        distance = haversine(*cities_info[k],*cities_info[goal_city])
        heuristics[k]= distance
    return heuristics

def calculate_total_distance(path, distances):
    """
    Calculate the total distance of the given path based on the road distances.
    
    :param path: A list of tuples representing the path (e.g., [('City1', 'City2'), ('City2', 'City3'), ...]).
    :param distances: A dictionary with city tuples as keys and distances as values.
    :return: The total distance of the path.
    """
    total_distance = 0
    for i in range(len(path) - 1):
        city_pair = (path[i], path[i+1])
        total_distance += distances.get(city_pair, 0)
    return total_distance

def fitness_function(path, distances):
    total_distance = calculate_total_distance(path, distances)
    if total_distance <= 0:
        return 0 
    return 1 / total_distance

# transition_cost_matrix = set_transition_and_cost_matrix(cities_info)

# import pandas as pd
# # Convert to DataFrame for pretty printing
# matrix_df = pd.DataFrame(transition_cost_matrix)

# # Fill diagonal with 0s for clarity (no distance to itself)
# for city in cities_info:
#     if city in matrix_df.index:
#         matrix_df.loc[city, city] = 0

# matrix_df_reordered = matrix_df.reindex(index=cities_info, columns=cities_info)

# matrix_df_reordered

# her = get_heuristics("Chennai")

# print(matrix_df_reordered," ",get_heuristics("Chennai"))


<h2> Supporting Functions </h2>

In [3]:
# Function to reconstruct the path from the start node to the goal node

# Function to keep track of path visited.
def reconstruct_path(came_from, start, goal):
    """
    Reconstructs the path from start to goal using the came_from dictionary.

    :param came_from: A dictionary containing the parent of each node
    :param start: The starting city
    :param goal: The goal city
    :return: A list of cities representing the path from the start to the goal
    """
    current = goal
    path = [current]
    while current != start:
        current = came_from[current]
        path.append(current)
    path.reverse()  # reverse the path to get the correct order from start to goal
    return path

# Example usage:
# came_from_example = {'Raichur': 'Panaji', 'Kurnool': 'Raichur', 'Chennai': 'Kurnool'}
# start_city = 'Panaji'
# goal_city = 'Chennai'
# reconstructed_path = reconstruct_path(came_from_example, start_city, goal_city)

# Function to display the path beautifully
def display_path_beautifully(path):
    """
    Displays the path in a structured textual representation.

    :param path: A list of cities representing the path from the start to the goal
    """
    for i in range(len(path)):
        if i == len(path) - 1:  # if it's the last city, just print the city
            print(path[i])
        else:  # otherwise, print the city and an arrow
            print(f"{path[i]} -> ", end="")

def is_path_valid(path, graph):
    for i in range(len(path) - 1):
        if path[i+1] not in graph[path[i]]:
            return False
    return True

def repair_path(path, start_city, goal_city, graph):
    repaired_path = [start_city]
    for current_city in path:
        if repaired_path[-1] == goal_city:
            break
        next_cities = graph.get(repaired_path[-1], [])
        if current_city in next_cities:
            repaired_path.append(current_city)
        else:
            # Find a valid next city that is not already in the repaired_path
            for possible_next in next_cities:
                if possible_next not in repaired_path:
                    repaired_path.append(possible_next)
                    break
    if repaired_path[-1] != goal_city:
        repaired_path.append(goal_city)  # Ensure the path ends at the goal city
    return repaired_path

# Display the path
# display_path_beautifully(reconstructed_path)

In [4]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

# State change function
def transition_model(current_city, graph):
    """
    Returns the successor states from the current city, along with the cost to reach each successor.

    :param current_city: The current city (state).
    :param graph: A dictionary representing the graph of cities, where keys are city names, and values are dictionaries
                  of neighboring cities and the distances to them.
    :return: A list of tuples, where each tuple contains a successor city and the cost to reach it.
    """
    successors = []
    if current_city in graph:
        for neighbor, cost in graph[current_city].items():
            successors.append((neighbor, cost))
    return successors

# Transition Model for Genetic Algorithm consist of Selection of Parents, CrossOver and Mutation of child.

# Function to select parents from population based their fitness score.
def selection(population, fitness_scores, selection_size):
    # Normalize fitness scores to use as probabilities for selection
    total_fitness = sum(fitness_scores)
    
    # Ensure total fitness is greater than zero and is finite
    if total_fitness <= 0 or not math.isfinite(total_fitness):
        raise ValueError("Invalid total fitness score: Must be greater than zero and finite.")
    
    selection_probabilities = [score / total_fitness for score in fitness_scores]
    
    # Select individuals based on their normalized fitness scores
    selected_indices = random.choices(
        range(len(population)), 
        weights=selection_probabilities, 
        k=selection_size
    )
    
    # Create a new list containing the selected individuals
    selected_population = [population[i] for i in selected_indices]
    return selected_population

#Function to generate the offspring/child from selected parents.
def crossover(parent1, parent2, cities):
    # Ensure paths are long enough for crossover
    min_length = min(len(parent1), len(parent2))
    if min_length <= 3:
        # If paths are too short, return them as is or handle differently
        return parent1, parent2

    # Proceed with crossover
    crossover_point = random.randint(1, min_length - 2)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]

    # Ensure the offspring are valid paths
    offspring1 = [city for city in cities if city in offspring1]
    offspring2 = [city for city in cities if city in offspring2]

    return offspring1, offspring2

#Function to generate strongest child by mutating offsprings/children. 
def mutate(path, mutation_rate):
    """
    Mutate a path by swapping two cities with a given probability.
    
    :param path: The path to mutate.
    :param mutation_rate: The probability of mutating a given gene.
    :return: The mutated path.
    """
    mutated_path = path.copy()
    for i in range(len(mutated_path)):
        if random.random() < mutation_rate:
            swap_index = random.randint(0, len(mutated_path) - 1)
            # Swap the cities at i and swap_index
            mutated_path[i], mutated_path[swap_index] = mutated_path[swap_index], mutated_path[i]
    return mutated_path

# Example usage
# current_city = 'Panaji'
# successors = transition_model(current_city, graph)
# print(f"Successors from {current_city}: {successors}")


In [5]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

# Goal Test Function for GBFS
def is_goal_reached(current_city, goal):
    """
    Test if the current city is the goal.

    :param current_city: The current city
    :param goal: The goal city
    :return: True if the current city is the goal, False otherwise
    """
    return current_city == goal

#Goal test for genetic algorithm.
def is_goal_reached_for_ga(path, goal_city):
    """
    Check if the goal city has been reached in the given path.
    
    :param path: A list representing the path taken, with each element being a city name.
    :param goal_city: The goal city name as a string.
    :return: True if the goal city is the last city in the path, False otherwise.
    """
    # Check if the last city in the path is the goal city
    return path[-1] == goal_city


### 2.	Definition of Algorithm 1 (Greedy BFS)

In [6]:
#Code Block : Function for algorithm 1 implementation


import heapq

# Modify the Greedy Best First Search Algorithm to include total cost
def greedy_best_first_search(graph, heuristics,initial_state):
    """
    Starts the GBFS algorithm using parameters defined in the initial state.

    :param graph: The graph of the problem.
    :param heuristics: Heuristic values for each city.
    :param initial_state: A dictionary containing the initial state parameters.
    :return: The results of the search.
    """
    start = initial_state['start_city']
    goal = initial_state['end_city']

    # Priority queue for the frontier cities, ordered by heuristic value
    frontier = []
    heapq.heappush(frontier, (heuristics[start], 0, start))  # includes current cost as 0

    # Dictionary to keep track of the path that led to each city
    came_from = {start: None}

    # Dictionary to keep track of the cost to reach each city
    cost_so_far = {start: 0}

    # The set of explored cities to avoid revisiting them
    explored = set()

    # Counters for nodes and edges explored
    nodes_explored = 0
    edges_explored = 0

    # GBFS loop
    while frontier:
        # Choose the city with the lowest heuristic value
        _, current_cost, current_city = heapq.heappop(frontier)

        nodes_explored += 1  # Increment nodes explored when a node is popped from the frontier

        if current_city == goal:
            return reconstruct_path(came_from, start, goal), current_cost, nodes_explored, edges_explored

        # Mark the city as explored
        explored.add(current_city)

        successors = transition_model(current_city, graph)

        # Add neighboring cities to the frontier if they haven't been explored or
        # if a cheaper path to them has been found
        # Use the transition_model function to get successors and their costs
        for neighbor, distance in successors:
            edges_explored += 1  # Increment edges explored for each neighbor examined
            new_cost = current_cost + distance
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                heapq.heappush(frontier, (heuristics[neighbor], new_cost, neighbor))
                came_from[neighbor] = current_city


    # If the goal was never reached, return None
    return None, None,nodes_explored, edges_explored


### 3.	Definition of Algorithm 2 (Genetic)

In [7]:
#Code Block : Function for algorithm 2 implementation

import random

def genetic_algorithm(cities, distances, population_size, generations, crossover_rate, mutation_rate, goal_city, start_city,graph,number_of_elites=1):
    """
    Implements a Genetic Algorithm with elitism to find the most optimal path to the goal city.
    
    :param cities: List of all cities in the problem.
    :param distances: Dictionary with distances between each pair of cities.
    :param population_size: Number of paths in each generation.
    :param generations: Maximum number of generations to evolve.
    :param crossover_rate: Probability of applying crossover.
    :param mutation_rate: Probability of applying mutation.
    :param goal_city: The destination city for the pathfinding problem.
    :param start_city: The start city for the pathfinding problem.
    :param graph: The map connetions between all cities.
    :param number_of_elites: The number of top individuals to carry over directly to the next generation.
    :return: The most optimal path found and its distance.
    """
    # Initialize the population
    population = set_initial_state_for_ga(start_city, goal_city, graph, population_size)
    # print("initial population ",population)
    best_path = None
    best_fitness = 0
    
    for generation in range(generations):
        # Evaluate the fitness of each individual in the population
        fitness_scores = [fitness_function(path, distances) for path in population]
    
        # Preserve the top individuals as elites
        elites_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i], reverse=True)[:number_of_elites]
        elites = [population[i] for i in elites_indices]
        
        # Selection (excluding elites if they are not to be mutated)
        parents = selection(population, fitness_scores, len(population) // 2)
        
        # Adjusting the parent list to ensure an even number for crossover
        if len(parents) % 2 != 0:
            # Remove the last parent if the count is odd
            parents.pop()
        
        # Crossover and Mutation
        offspring = []
       
        for _ in range(0, len(parents), 2):
            if random.random() < crossover_rate:
                child1, child2 = crossover(parents[_], parents[_+1], cities)
            else:
                child1, child2 = parents[_], parents[_+1]

            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            if not is_path_valid(child1, graph):
                child1 = repair_path(child1, start_city, goal_city, graph)
            if not is_path_valid(child2, graph):
                child2 = repair_path(child2, start_city, goal_city, graph)

            offspring.extend([child1, child2])
        
        # Combine elites with offspring to form the new generation
        new_population = elites + offspring[:population_size - len(elites)]
        population = new_population
        
        # Update best path if a new best is found
        for elite in elites:
            elite_fitness = fitness_function(elite, distances)
            if is_goal_reached_for_ga(elite, goal_city) and elite_fitness > best_fitness:
                best_path = elite
                best_fitness = elite_fitness
        
        # print(f"Generation {generation + 1}: Best Path Distance = {1 / best_fitness if best_fitness else 'Not found'}")
    
    return best_path, 1 / best_fitness if best_fitness else None

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [8]:
#Code Block : Function & call to get inputs (start/end state)
import time 
def get_start_end_states(cities):
    """
    Prompt the user to select the start and end states for the journey by number instead of typing names.

    :param cities: A list of available cities.
    :return: A tuple containing the start and end states.
    """
    # Display cities with numbers
    for i, city in enumerate(cities, 1):
        print(f"{i}. {city}")

    
    time.sleep(2)

    while True:
        try:
            start_index = int(input("Select the number for the start city: ")) - 1
            if start_index not in range(len(cities)):
                print("Invalid selection. Please select a number from the list.")
                continue

            end_index = int(input("Select the number for the destination city: ")) - 1
            if end_index not in range(len(cities)):
                print("Invalid selection. Please select a number from the list.")
                continue

            if start_index == end_index:
                print("Start and destination cities cannot be the same. Please select different cities.")
                continue

            start = cities[start_index]
            end = cities[end_index]

            return start, end
        except ValueError:
            print("Invalid input. Please enter a number.")


# Call the function to get start and end states
# start_city, end_city = get_start_end_states(cities_list)
# print(f"Starting from: {start_city}, Destination: {end_city}")


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [9]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Call the function to get start and end states
start_city, end_city = get_start_end_states(cities_list)

# Prepare initial state
initial_state = set_initial_state(start_city, end_city)
# time & space of gbfs
import time
start_time = time.perf_counter()
# Implement GBFS with cost
path_gbfs, total_cost_gbfs, node_explored,edge_explored = greedy_best_first_search(graph, get_heuristics(end_city),initial_state)
end_time = time.perf_counter()


# Display the path and total cost if a path was found
if path_gbfs:
    display_path_beautifully(path_gbfs)
    print(f"Total cost: {total_cost_gbfs} km")

else:
    print("No path found from Panaji to Chennai using GBFS.")


1. Panaji
2. Raichur
3. Mangalore
4. Bellari
5. Tirupathi
6. Kurnool
7. Kozhikode
8. Bangalore
9. Nellore
10. Chennai


Panaji -> Raichur -> Tirupathi -> Chennai
Total cost: 1063 km


In [10]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

start_city, goal_city = get_start_end_states(cities_list)
import time
start_time_ga = time.perf_counter()
population_size = 200

generations = 100
crossover_rate = 0.8
mutation_rate = 0.2

# Invoke the Genetic Algorithm
best_path, best_distance = genetic_algorithm(cities_list, distances, population_size, generations, crossover_rate, mutation_rate, goal_city, start_city,graph)
end_time_ga = time.perf_counter()
# Print the solution
if best_path and best_distance:
    print(f"Solution Found from Genetic Algorithm:\nPath: {' -> '.join(best_path)}\nCost (Total Distance): {best_distance} km")
else:
    print("No solution found within the given constraints.")

1. Panaji
2. Raichur
3. Mangalore
4. Bellari
5. Tirupathi
6. Kurnool
7. Kozhikode
8. Bangalore
9. Nellore
10. Chennai
Solution Found from Genetic Algorithm:
Path: Panaji -> Raichur -> Kurnool -> Nellore -> Chennai
Cost (Total Distance): 1057.0 km


### 5.	Comparitive Analysis

In [11]:
#Code Block : Print the Time & Space complexity of algorithm 1

def calculate_average_branching_factor(graph):
    total_successors = sum(len(neighbors) for neighbors in graph.values())
    num_nodes = len(graph)
    return total_successors / num_nodes
average_branching_factor = calculate_average_branching_factor(graph)
print("Average Branching Factor:", average_branching_factor)

space_complexity = pow(average_branching_factor,node_explored)
print(f"Estimated Time Complexity & Space Complexity for GBFS from {initial_state['start_city']} to {initial_state['end_city']}:")
print(f"Space Complexity O(b^d): {space_complexity}")
print("Time complexity : %.6f seconds" % (end_time - start_time))



Average Branching Factor: 2.6
Estimated Time Complexity & Space Complexity for GBFS from Panaji to Chennai:
Space Complexity O(b^d): 45.69760000000001
Time complexity : 0.000107 seconds


In [12]:
#Code Block : Print the Time & Space complexity of algorithm 2
# The dominating terms in the space complexity are from the population size and number of cities.

print(f"Estimated Time Complexity & Space Complexity for Genetic Algorithm from {start_city} to {goal_city}:")
print(f"Space Complexity O(b^d): {population_size * len(cities_list)}")
print("Time complexity : %.6f seconds" % (end_time_ga - start_time_ga))

Estimated Time Complexity & Space Complexity for Genetic Algorithm from Panaji to Chennai:
Space Complexity O(b^d): 2000
Time complexity : 0.002009 seconds


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : GBFS vs Genetic Algorithm

1. <b>Optimality</b>: GBFS may not always find the optimal solution, especially if the heuristic function is not accurate or the search space is complex. On the other hand, GA has the potential to find better solutions through evolutionary processes but may take longer to converge.<br>
GBFS --> Panaji -> Raichur -> Tirupathi -> Chennai | Total cost: 1063 km <br>
GA -->   Panaji -> Raichur -> Kurnool -> Nellore -> Chennai | Total Cost : 1057.0 km
2. <b>Efficiency</b>: GBFS is generally more efficient in terms of time complexity for finding a single shortest path, but it may not perform well in complex search spaces or when the heuristic function is not well-suited. GA, although slower, can explore a larger portion of the search space and potentially find better solutions.
GBFS - Time complexity : 0.000112 seconds vs GA - Time complexity : 0.002497 seconds
3. <b>Resource Usage</b>: GBFS typically requires less memory compared to GA, as it only needs to store information about the search frontier and the explored nodes. GA, on the other hand, requires storing a population of potential solutions, which can consume more memory.